In [9]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-06-27 14:50:38--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-06-27 14:50:38 (14.0 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [5]:
import minsearch

In [8]:
!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json

--2024-06-28 09:14:11--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 658332 (643K) [text/plain]
Saving to: ‘documents.json’

documents.json      100%[===================>] 642.90K  --.-KB/s    in 0.01s   

2024-06-28 09:14:11 (52.5 MB/s) - ‘documents.json’ saved [658332/658332]



In [6]:
import json

# Preparing the FAQ Documents

The FAQ documents were parsed by Alexey previousle using the parse-faq.ipynb notebook. There were 3 FAQ documents, one for each course. For each course, we will have a json object with documents inside. This is all in a json file called documents.json

In [9]:
#this will be opened in read text mode
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [10]:
#let's remove the nextedness

documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [11]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [19]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [20]:
SELECT * WHERE course = 'data-engineering-zoomcamp'; 

SyntaxError: invalid syntax (1083679446.py, line 1)

In [21]:
q = 'the course has already started, can I still enroll?'

In [22]:
index.fit(documents)

In [23]:
boost = {'question': 3.0, 'section':0.5}

results = index.search(
    query=q,
    boost_dict=boost,
    num_results=5
)

In [24]:
results

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the cour

In [27]:
from openai import OpenAI

In [28]:
client = OpenAI(api_key='sk-llm-zoomcamp-g1qn5cqLe1Gp0s0wLdZ9T3BlbkFJvHgr66ALUPaOsuWGi8ef')

# Response analysis

In [30]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": q}]
)

response.choices[0].message.content

"Whether or not you can still enroll in a course that has already started depends on the policies of the institution or organization offering the course. Here are some general steps you can take:\n\n1. **Check the Course Details:** Look for any specific information about late enrollment in the course syllabus or on the course webpage.\n\n2. **Contact the Instructor:** Reach out directly to the course instructor to explain your situation and ask if it's still possible to join the class.\n\n3. **Contact the Registrar/Administration:** If the course is offered through a university or another formal institution, the registrar's office or academic administration may have policies regarding late enrollment.\n\n4. **Online Courses:** Many online learning platforms like Coursera, Udemy, or edX are more flexible with enrollment dates. Check if the course offers a self-paced option or rolling enrollment.\n\n5. **Catch Up:** If you are allowed to enroll late, be prepared to catch up on any missed

In [31]:
response

ChatCompletion(id='chatcmpl-9f2gxOyVnEmpWC510gFqrSjfYe3Os', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Whether or not you can still enroll in a course that has already started depends on the policies of the institution or organization offering the course. Here are some general steps you can take:\n\n1. **Check the Course Details:** Look for any specific information about late enrollment in the course syllabus or on the course webpage.\n\n2. **Contact the Instructor:** Reach out directly to the course instructor to explain your situation and ask if it's still possible to join the class.\n\n3. **Contact the Registrar/Administration:** If the course is offered through a university or another formal institution, the registrar's office or academic administration may have policies regarding late enrollment.\n\n4. **Online Courses:** Many online learning platforms like Coursera, Udemy, or edX are more flexible with enrollment dates. Ch

## Response was very generic, as expected, so now we want to get a response with more context. For that we will do a bit of prompt engineering, where we will direct the model toward some sort of behaviour, and by asking it to use a specific context to answer the question. The context in this case will be the query we performed on the indexed FAQ documents

In [32]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION. 

QUESTION: {question}

CONTEXT: 
{context}
"""

In [36]:
context = ""

for doc in results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [37]:
print(context)

section: General course-related questions
question: The course has already started. Can I still join it?
answer: Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.
In order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.

section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you c

In [38]:
prompt = prompt_template.format(question=q, context=context).strip()

In [39]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": prompt}]
)

response.choices[0].message.content

"Yes, you can still enroll in the course even if it has already started. However, you may not be able to submit some of the homeworks. To be eligible for a certificate, you need to submit 2 out of 3 course projects and review 3 peers' projects by the deadline. Even if you join later, you can still participate actively and potentially earn the certificate if you meet these requirements."